# 합성곱 신경망 소개

In [2]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [3]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Conv2D와 MaxPooling2D 층의 출력은 (height, width, channels) 크기의 3D 텐서이다. 높이와 너비 차원은 네트워크가 깊어질수록 작아지는 경향이 있다.

다음 단계에서 마지막 층의 ((3, 3, 64) 크기인) 출력 텐서를 완전 연결 네트워크에 주입한다. 이 네트워크는 이미 익숙하게 보았던 Dense 층을 쌓은 분류기이다. 이 분류기는 !D 벡터를 처리하는데, 이전 층의 출력이 3D 텐서이기 때문에, 먼저 3D 출력을 1D 텐서로 펼쳐야 한다. 그 다음 몇 개의 Dense 층을 추가한다.

In [4]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [5]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

In [9]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 62s 1ms/step - loss: 0.1708 - accuracy: 0.9468
Epoch 2/5
60000/60000 [==============================] - 61s 1ms/step - loss: 0.0463 - accuracy: 0.9858
Epoch 3/5
60000/60000 [==============================] - 61s 1ms/step - loss: 0.0316 - accuracy: 0.9905
Epoch 4/5
60000/60000 [==============================] - 62s 1ms/step - loss: 0.0242 - accuracy: 0.9926
Epoch 5/5
60000/60000 [==============================] - 61s 1ms/step - loss: 0.0201 - accuracy: 0.9940


In [13]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 4s 385us/step


In [14]:
test_acc

0.9919999837875366

완전 연결 층과 합성곱 사이의 근본적인 차이는 다음과 같다. Dense 층은 입력 특성 공간에 있는 전역 패턴(예를 들어 MNIST 숫자 이미지는 모든 픽셀에 걸친 패턴)을 학습하지만 합성곱 층은 지역 패턴을 학습한다. 이 핵심 특징은 컨브넷에 두 가지 흥미로운 성질을 제공한다.
* 컨브넷이 이미지의 오른쪽 아래 모서리에서 어떤 패턴을 학습했다면 다른 곳(예를 들어 왼쪽 위 모서리)에서도 이 패턴을 인식할 수 있다.
* 컨브넷은 패턴의 공간적(부분적) 계층 구조를 학습할 수 있다.

In [18]:
model_no_max_pool = models.Sequential()

model_no_max_pool.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu'))

스트라이드 합성곱과 매우 비슷하게 강제적으로 특성 맵을 다운샘플링 하는 것이 최대 풀링의 역할이다.

In [19]:
model_no_max_pool.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 22, 22, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


이 설정에서 두 가지 문제가 있다.

1. 컨브넷에 의해 학습관 고수준 패턴은 초기 입력에 관한 정보가 아주 적어 숫자 분류를 학습하기에 충분하지 않다.
2. 최종 특성 맵은 22x22x64 = 30,976개의 원소를 가진다. 아주 많다. 이 컨브넷을 펼친 후 512 크기의 Dense 층과 연결한다면 약 15.8백만 개의 가중치 파라미터가 생긴다. 작은 모델 치고 너무 많은 가중치이고, 심각한 과대적합이 발생할 것이다.

간단히 말해 다운샘플링을 사용하는 이유는 처리할 특성 맵의 가중치 개수를 줄이기 위해서이다.

# 소규모 데이터셋에서 밑바닥부터 컨브넷 훈련하기